In [2]:
%matplotlib inline
import matplotlib
import seaborn as sns
sns.set()
matplotlib.rcParams['figure.dpi'] = 144

# PySpark MLlib
<!-- requirement: small_data/gutenberg -->

*Official documentation [here](https://spark.apache.org/docs/latest/mllib-guide.html).*

In [3]:
from pyspark import SparkContext
sc = SparkContext("local[*]", "temp")
print(sc.version)

2.3.1


In [4]:
# needed to convert RDDs into DataFrames
from pyspark.sql import SQLContext
sqlContext = SQLContext(sc)

In [5]:
# Tells Spark to look on the local filesystem
import os
def localpath(path):
    return 'file://' + str(os.path.abspath(os.path.curdir)) + '/' + path

## Algorithms


Spark supports a number of machine-learning algorithms.

- Classification and Regression
    - SVM, linear regression
    - SVR, logistic regression
    - Naive Bayes
    - Decision Trees
    - Random Forests and Gradient-Boosted Trees
- Clustering
    - K-means (and streaming K-means)
    - Gaussian Mixture Models
    - Latent Dirichlet Allocation
- Dimensionality Reduction
    - SVD and PCA
- It also has support for lower-level optimization primitives:
    - Stochastic Gradient Descent
    - Low-memory BFGS and L-BFGS

### Parallelized SGD


For linear models like SVM, Linear Regression, and Logistic Regression, the cost function we're trying to optimize is essentially an average over the individual error term from each data point. This is particularly great for parallelization.  For example, in linear regression, recall that the gradient is

$$\begin{align}
\frac{\partial \log(L(\beta))}{\partial \beta} &= \frac{\partial}{\partial \beta} \frac{1}{2}\sum_j \|y_j - X_{j \cdot} \cdot \beta\| \\
&= \frac{1}{2}\sum_j \frac{\partial}{\partial \beta} \|y_j - X_{j \cdot} \cdot \beta\| \\
& = \sum_j y_j - X_{j \cdot} \cdot \beta \\
& \approx \sum_{sample \mbox{ } j} y_j - X_{j \cdot} \cdot \beta
\end{align}$$

The key *mathematical properties* we have used are:

1. the error functions are the sum of error contributions of different training instances
1. linearity of the derivative
1. associativity of addition
1. downsampling giving an unbiased estimator

Since the last sum is over the different training instances and these are stored on different nodes, we can parallelize the computation of the gradient in SGD across multiple nodes.  Of course, we still need to maintain the running weight $\beta$ that has to be present on every node (through a broadcast variable that is updated).  Notice that SVM, Linear Regression, and Logistic Regression all have error functions that are just sums over training instances so SGD can be used for all these algorithms.

Spark's [implementation](http://spark.apache.org/docs/latest/mllib-optimization.html#stochastic-gradient-descent-sgd) uses a tunable mini-batch size parameter to sample a percentage of the features RDD. For each iteration, the updated weights are broadcast to the executors, and the update is calculated for each data point and sent back to be aggregated.

Parallelization handles increasing number of sampled data points m quite well since there are no interaction terms and each calculation is independent. Controlling how the algorithm iterates to convergence is also important, and can be done with parameters for the total iterations and step size.

## ML vs. MLlib packages


Confusingly, there are two machine learning APIs in Spark, the `mllib` package based on RDDs and the `ml` package based on DataFrames. For years these have been developed somewhat in parallel, resulting in duplication and asymmetry in functionality.

With Spark 2.0+, `mllib` is in maintenance mode and will likely be deprecated in future in favor of the DataFrame-based API which more closely resembles libraries like Scikit-learn. Below is one example of the RDD-based API; the rest of the notebook will focus on DataFrames.

In [6]:
from pyspark.mllib.regression import LinearRegressionWithSGD, LinearRegressionModel, LabeledPoint
from pyspark.mllib.evaluation import RegressionMetrics
from pyspark.mllib.linalg import Vector, Vectors
import random

# parameters
TRAINING_ITERATIONS = 10
TRAINING_FRACTION = 0.6

# generate the data
data = sc.parallelize(range(1,10001)) \
    .map(lambda x: LabeledPoint(random.random(), [random.random(), random.random(), random.random()]))

# split the training and test sets
splits = data.randomSplit([TRAINING_FRACTION, 1 - TRAINING_FRACTION], seed=42)
training, test = (splits[0].cache(), splits[1])

# train the model
model = LinearRegressionWithSGD.train(training, iterations=TRAINING_ITERATIONS)

# get r2 score
predictions = test.map(lambda x: (float(model.predict(x.features)), x.label))
print(RegressionMetrics(predictions).r2)

-0.30032448230133535


Maybe we can improve this by modeling the intercept. Use `<shift-tab>` inside the arguments to bring up the docstring for `LinearRegressionWithSGD`, and rerun the training with an intercept term.

If you're interested in methods for introspecting some of these objects, the `<tab>` and `<shift-tab>` documentation is good. You can also use `dir` in Python to list all the components of something.

In [7]:
dir(test.take(2)[0])

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 'features',
 'label']

## Spark ML

Spark ML implements the ideas of transformers, estimators, and pipelines by standardizing APIS across machine learning algorithms. This can streamline more complex workflows.

The core functionality includes:
* DataFrames - built off Spark SQL, can be created either directly or from RDDs as seen above
* Transformers - algorithms that accept a DataFrame as input and return a DataFrame as output
* Estimators - algorithms that accept a DataFrame as input and return a Transformer as output
* Pipelines - chaining together Transformers and Estimators
* Parameters - common API for specifying hyperparameters

For example, a learning algorithm can be implemented as an Estimator which trains on a DataFrame of features and returns a Transformer which can output predictions based on a test DataFrame.

Full documentation can be found [here](http://spark.apache.org/docs/latest/ml-guide.html).

Lets start with a simple example where we want to classify reviews as either good (1.0) or bad (0.0).  For this example, lets create some fake reviews we can use for training.  We will use both a training set `reviews` and a testing set `test_reviews` towards this purpose.  While this is not enough reviews to train any sort of decent model, its enough to illustrate Spark machine learning.

In [8]:
reviews = [("Prose is well-written, but style is an impediment to learning. Should be called 'Reviewing Spark,' not 'Learning Spark'", 0.0),
            ("Nice Headstart to Spark", 1.0),
            ("Start here: Excellent reference for Spark", 1.0),
            ("Insightful and so Spark-tastic!", 1.0),
            ("Good intro but wordy and lacking details in areas", 0.0),
            ("Best of the Books Currently Available", 1.0),
            ("A good resource for people interested in learning Spark", 1.0),
            ("Great Overview", 1.0)]

test_reviews = [("A decent guided tour of Spark and its major components.", 0.0),
                ("10/10 would buy again", 1.0),
                ("it is simple to follow. well organized. straight ...", 1.0),
                ("Just what you need to get started in Apache Spark.", 1.0),
                ("Very good book for learning Spark", 1.0)]

For those coming from `scikit-learn` or other packages, one of the major differences in the data paradigm is the immutability of DataFrames.  Unlike in `scikit-learn` where in some sense the `Transformer` object replaces the feature matrix at each transformation step, the Spark `Transformer` takes as input a DataFrame and outputs a new DataFrame where a column has been added containing the transformed features. To see this in action, lets first import the necessary libraries from `pyspark.ml` and then create our DataFrames from the review lists.  Note that we `cache` the training data, but not the test data,  why?

In [9]:
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.feature import HashingTF, Tokenizer

training = sqlContext.createDataFrame(reviews, ["title", "label"]).cache()
test = sqlContext.createDataFrame(test_reviews, ["title", "label"])

Now let us create the `Transformers` and `Estimators` we need for the training.  We will use two `Transformers`, a `Tokenizer` to tokenize the words and then a `HashingTF` to hash these words into a feature matrix (effectively a word count with a hash function).  Lastly, we will use a `LogisticRegression` `Estimator` to perform the logistic regression classification.  

Each of these `ML` objects take both an `inputCol` and an `outputCol` keyword argument (they have reasonably sane defaults).  When their transform method is called, they take the input `DataFrame` and produce a `DataFrame` which has the extra column `outputCol` with the relevant transformation applied to `inputCol`. 

An `Estimator` has a `fit` method, the return value of this method is itself a `Transformer`, one which takes an input column and outputs a `DataFrame` with a column of predictions.  Lets fit our model.

In [10]:
tokenizer = Tokenizer(inputCol="title", outputCol="words")
hashingTF = HashingTF(inputCol=tokenizer.getOutputCol(), outputCol="features")
logreg = LogisticRegression(maxIter=10, regParam=0.01)

tokens = tokenizer.transform(training)
hashes = hashingTF.transform(tokens)
model = logreg.fit(hashes)

Now we can make predictions on our test data.  Note that we do need to perform the same transformations, in this case in the same order (although not always, when would this not be necessary?) before we can call the `transform` method of the fitted model.  Once we get a prediction, we can select out the particular columns of the `DataFrame` we care about.

In [11]:
# Make predictions on test documents
test_tokens = tokenizer.transform(test)
test_hashes = hashingTF.transform(test_tokens)

prediction = model.transform(test_hashes)
selected = prediction.select("title", "label", "prediction")
for row in selected.collect():
    print(row)

Row(title='A decent guided tour of Spark and its major components.', label=0.0, prediction=1.0)
Row(title='10/10 would buy again', label=1.0, prediction=1.0)
Row(title='it is simple to follow. well organized. straight ...', label=1.0, prediction=1.0)
Row(title='Just what you need to get started in Apache Spark.', label=1.0, prediction=1.0)
Row(title='Very good book for learning Spark', label=1.0, prediction=1.0)


In [12]:
# Note that if you use a PipelineModel it won't have a coefficients attribute.
model.coefficients

SparseVector(262144, {6368: -0.4672, 9639: 0.4928, 15889: -0.2336, 16332: 0.5375, 22346: 0.7979, 26114: -0.4672, 29952: -1.0939, 39221: 0.4377, 47372: 0.4928, 47822: 0.8852, 48390: 0.7979, 61971: 0.4928, 77044: -0.4672, 81677: -0.4672, 91677: -0.1217, 99585: -0.4672, 103838: 0.4928, 104587: 0.4377, 106119: -0.4672, 113432: -0.356, 114686: 0.4837, 117481: 0.4377, 123535: -1.0939, 136358: -0.4672, 138356: 1.1303, 138912: -1.0939, 139098: -0.4672, 147278: -0.4672, 149001: -0.4672, 149910: 0.4837, 163984: 0.4837, 166027: 0.4928, 167152: -0.4672, 175563: 1.1303, 178534: -0.4672, 188424: 0.8852, 189683: -0.9106, 194536: -0.4672, 205044: 0.1929, 212740: 0.4377, 221048: -1.0939, 222453: -0.356, 227410: 0.4837, 234657: 0.8024, 235528: 0.4837, 238163: 0.4928, 241199: 0.8852, 247066: -1.0939, 261451: -0.4672})

In [13]:
prediction.select(["features", "probability", "prediction"]).show(2)

+--------------------+--------------------+----------+
|            features|         probability|prediction|
+--------------------+--------------------+----------+
|(262144,[9639,160...|[0.00839737493866...|       1.0|
|(262144,[68867,70...|[0.04252139879146...|       1.0|
+--------------------+--------------------+----------+
only showing top 2 rows



## Pipeline

Its been a bit annoying to make sure to pass the output from `tokenizer` into the input of the `hashingTF`, its also a bit error prone since it must be done on the inference step as  well.  Luckily `Spark ML` offers a nice `Pipeline` abstraction to build data processes in a simple, easy way.  A `Pipeline` takes in multiple *stages* of `Transformer` and `Estimator` and is itself an `Estimator`, meaning it has `fit` and `transform` methods.  When the `fit` method is called, the `transform` method of `Transformer`s and the `fit` method of `Estimator`s are called.  The returned `PipelineModel` has all transformers (each `Estimator`'s `fit` method returns a `Transformer`).  If you are used to `scikit-learn` this is similar in spirit to a `sklearn.pipeline.Pipeline`, but different in approach.

These constructs are immensely useful for any sort of ML task and should almost always be used in any circumstance where more than a single `Estimator` is used (which is almost always the case).

In [14]:
from pyspark.ml import Pipeline
pipeline = Pipeline(stages=[tokenizer, hashingTF, logreg])

*Question*: For those coming from `scikit-learn`, what is the equivalent of a `FeatureUnion`?*

## Cross-validation and grid search

Tuning hyperparameters through cross validation is necessary for the most performant models.  While doing this grid search by hand is not difficult, there is a perfectly good high level model already built in.  We can use a `ParamGridBuild` object to build a grid to search over.  Each `Transformer` has a number of hyperparameters we can tune, and we can add a grid for as few or as many of these as we would like.

In [15]:
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from pyspark.ml.evaluation import BinaryClassificationEvaluator

In [16]:
paramGrid = (ParamGridBuilder() 
    .addGrid(hashingTF.numFeatures, [100, 1000]) 
    .addGrid(logreg.regParam, [0.1, 0.01]) 
    .build())

Now that we have a `paramGrid`, we can perform cross validation over each hyperparameter in the grid.  We do this with a `CrossValidator` object which takes in an `Estimator` (in this case our pipeline), a set of parameters, an evaluator, and the number of folds.  We will use a `BinaryClassificationEvaluator` here because we are in fact doing binary classification (its so nice when things make sense).

In [17]:
crossval = CrossValidator(estimator=pipeline,
                          estimatorParamMaps=paramGrid,
                          evaluator=BinaryClassificationEvaluator(),
                          numFolds=3)

*Note*: A more traditional validation set without folding is available in `TrainValidationSplit`.

The `CrossValidator` object is itself an `Estimator`, thus it can be fit to produce a `Transformer` than can make predictions.

In [18]:
cvModel = crossval.fit(training)

In [19]:
better_prediction = cvModel.transform(test)

In [20]:
selected = better_prediction.select("title", "label", "prediction")
for row in selected.collect():
    print(row)

Row(title='A decent guided tour of Spark and its major components.', label=0.0, prediction=1.0)
Row(title='10/10 would buy again', label=1.0, prediction=1.0)
Row(title='it is simple to follow. well organized. straight ...', label=1.0, prediction=1.0)
Row(title='Just what you need to get started in Apache Spark.', label=1.0, prediction=1.0)
Row(title='Very good book for learning Spark', label=1.0, prediction=1.0)


There are some nice introspection methods available, like the best model, as well as the attributes of that model as long as you are willing to dig deep enough into the stages.

In [21]:
cvModel.bestModel.stages

[Tokenizer_4bd6977fbc87eb77837d,
 HashingTF_4dcc8317039795b955c2,
 LogisticRegression_4315b6d40f73e42eec38]

In [22]:
cvModel.bestModel.stages[2].coefficients

SparseVector(100, {0: 0.1736, 3: 0.2509, 5: 0.2582, 8: -0.4224, 9: 0.1177, 10: -0.1452, 11: 0.1546, 14: -0.8297, 18: -0.4224, 21: 0.1177, 26: 0.2509, 28: -0.8297, 33: -0.3376, 34: -0.4224, 35: -0.4224, 36: 0.1721, 39: -0.8297, 43: 0.1736, 45: -0.3806, 51: 0.1177, 52: -0.2112, 56: -0.4224, 57: -0.4224, 60: 0.1546, 62: -0.4224, 63: 0.0868, 65: -0.2112, 68: -0.1874, 70: 0.1271, 71: -0.1452, 72: 0.1773, 76: -0.8297, 81: -0.2112, 83: -0.7304, 84: 0.1773, 86: 0.1773, 88: -0.4638, 96: 0.1177, 98: -0.4224})

In [23]:
training.unpersist()

DataFrame[title: string, label: double]

### Example algorithm: `Word2Vec`

`Word2Vec` is a model which embeds words in a vector space using a `skip-gram` model.  The idea is to capture in the meaning of word both in an absolute sense and in a contextual sense as a vector.  The model allows one to take a document and turn it into a vector which is the average vector of all words in the document.  This can be used for prediction and also for similarity as will be shown here.

In [24]:
from pyspark.ml.feature import Word2Vec

gutenberg = sc.textFile(localpath('small_data/gutenberg/')).map(lambda line: (line.split(" "), 1)).toDF(['text', 'score'])
w2v = Word2Vec(inputCol="text", outputCol="vectors", vectorSize=30, minCount=10, seed=17)
model = w2v.fit(gutenberg)
result = model.transform(gutenberg)

In [25]:
vectors = model.getVectors().rdd.map(lambda x: (x.word, x.vector))
print(model.findSynonyms('woman', 10).rdd.take(10))

[Row(word='bit', similarity=0.8855902552604675), Row(word='nun', similarity=0.8483726382255554), Row(word='fine', similarity=0.8426378965377808), Row(word='nice', similarity=0.8423601388931274), Row(word='few', similarity=0.8347029685974121), Row(word='good', similarity=0.8334241509437561), Row(word='married', similarity=0.831967830657959), Row(word='woman.', similarity=0.8261054754257202), Row(word='fellow', similarity=0.8226621150970459), Row(word='big', similarity=0.8077748417854309)]


In [26]:
king_vec = vectors.lookup('king')[0]
queen_vec = vectors.lookup('queen')[0]
man_vec = vectors.lookup('man')[0]
woman_vec = vectors.lookup('woman')[0]

In [27]:
print(king_vec)

[-0.11059810221195221,0.04715776443481445,-0.057438697665929794,0.0370696596801281,0.059630125761032104,0.11818759143352509,0.006231487728655338,-0.06164010614156723,0.03297163173556328,-0.03823459520936012,-0.03562337905168533,0.0256644394248724,0.0008936242666095495,-0.17509834468364716,-0.005442800000309944,0.07037258893251419,0.040936607867479324,-0.005964596290141344,0.05871565639972687,-0.02230997197329998,-0.06660382449626923,-0.003575243055820465,0.09434086084365845,0.08188135176897049,-0.0030699509661644697,0.05123915895819664,-0.11344990879297256,0.027308732271194458,0.1091141402721405,-0.011432445608079433]


In [28]:
print(queen_vec.squared_distance(king_vec))
print(queen_vec.squared_distance(woman_vec))
print(queen_vec.squared_distance(man_vec))
print(queen_vec.squared_distance(king_vec + man_vec - woman_vec))
print(queen_vec.squared_distance(king_vec - man_vec + woman_vec))

0.14436983499684541
1.3814033584587042
1.3918323033622386
0.8321534483610576
0.7671231576265367


## Feature processing

Often after we do some sort of feature extraction, we want to do further processing on the features.  Two common cases would be to select only certain features in a particular column, or to combine multiple columns into a single column to pass into an `Estimator`.  Remember, most `Estimator`s take only a single column as the input features.

Lets start with a `DataFrame` of two columns.

In [ ]:
vdf = vectors.toDF(["word", "vector"])

In [ ]:
vdf.show(5)

In [ ]:
sample_vector = vdf.select("vector").take(1)[0][0]
print(len(sample_vector))

Each of the vector column has 100 elements, but what if we only want a few of these elements?  This is where the `VectorSlicer` can be helpful, it allows us to select only some indices of a particular vector.

In [ ]:
from pyspark.ml.feature import VectorSlicer

first_slicer = VectorSlicer(inputCol="vector", outputCol="first", indices=[0])
last_slicer = VectorSlicer(inputCol="vector", outputCol="last", indices=[len(sample_vector) - 1])
med_slicer = VectorSlicer(inputCol="vector", outputCol="med", indices=range(15, 25))

Since `DataFrame`s are immutable, we can pass the result of each of these `VectorSlicer` objects into the next one (or we could use a pipeline).

In [ ]:
output = med_slicer.transform(last_slicer.transform(first_slicer.transform(vdf)))
output.columns

Now we have three columns of features, let us combine them together.  To do this, we can use the `VectorAssembler` which takes multiple columns and makes them into one.  Often this will come after a series of various transformation on different parts of the data before fitting the final model.  Again, how can we reproduce a `scikit-learn` `FeatureUnion` with these tools?

In [ ]:
from pyspark.ml.feature import VectorAssembler

assembler = VectorAssembler(
    inputCols=["first", "last", "med"],
    outputCol="features")

new_output = assembler.transform(output)
new_output.columns

In [ ]:
from pyspark.ml.feature import Binarizer

binarizer = Binarizer(threshold=0.05, inputCol="features", outputCol="bin_features")
new_output_b = binarizer.transform(new_output)
print(new_output_b.select(["features", "bin_features"]).take(1)[0])

### Exercise: Use SVM to predict colon cancer from gene expressions

You can start getting a feel for the MLlib operations by following the [Spark docs example](https://spark.apache.org/docs/1.3.0/mllib-linear-methods.html#linear-support-vector-machines-svms) on this data set.


#### About the data format: LibSVM

MLlib conveniently provides a data loading method, `MLUtils.loadLibSVMFile()`, for the LibSVM format for which many other languages (R, MATLAB, etc.) also have loading methods.  
A data set of *n* features will have one row per datum, with the label and values of each feature organized as follows:
>{label} 1:{value} 2:{value} ... n:{value}

Take these two data points with six features and labels of -1 and 1 respectively as an example:
>-1.000000  1:2.080750 2:1.099070 3:0.927763 4:1.029080 5:-0.130763 6:1.265460  
1.000000  1:1.109460 2:0.786453 3:0.445560 4:-0.146323 5:-0.996316 6:0.555759 


#### About the colon-cancer data set

This data set was introduced in the 1999 paper "Broad patterns of gene expression revealed by clustering analysis of tumor and normal colon tissues probed by oligonucleotide arrays." (Available on PNAS)

Here's the abstract of the paper:  
> *Oligonucleotide arrays can provide a broad picture of the state of the cell, by monitoring the expression level of thousands of genes at the same time. It is of interest to develop techniques for extracting useful information from the resulting data sets. Here we report the application of a two-way clustering method for analyzing a data set consisting of the expression patterns of different cell types. Gene expression in 40 tumor and 22 normal colon tissue samples was analyzed with an Affymetrix oligonucleotide array complementary to more than 6,500 human genes. An efficient two-way clustering algorithm was applied to both the genes and the tissues, revealing broad coherent patterns that suggest a high degree of organization underlying gene expression in these tissues. Coregulated families of genes clustered together, as demonstrated for the ribosomal proteins. Clustering also separated cancerous from noncancerous tissue and cell lines from in vivo tissues on the basis of subtle distributed patterns of genes even when expression of individual genes varied only slightly between the tissues. Two-way clustering thus may be of use both in classifying genes into functional groups and in classifying tissues based on gene expression.*

There are 2000 features, 62 data points (40 tumor (label=0), 22 normal (label=1)), and 2 classes (labels) for the colon cancer data set. 

#### Exit Tickets

1. When would you use `org.apache.spark.mllib.linalg.Vector` versus `breeze.linalg.DenseVector`?
1. Why can SVM, Linear Regression, and Logistic Regression be parallelized?  How would you parallelize KMeans?


In [ ]:
sc.stop()

*Copyright &copy; 2018 The Data Incubator.  All rights reserved.*